<a href="https://colab.research.google.com/github/ardywibowo/cuda-mode/blob/main/chapter3/chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3

## Setup

In [1]:
!pip install ninja
!sudo apt update
!sudo apt install g++-11 -y
!sudo apt install ccache -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 6.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [670 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,408 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,704 kB]
Hit:12 

In [2]:
import torch
import torch.utils.cpp_extension
import os
os.environ['CXX'] = '/usr/lib/ccache/g++-11'
os.environ['CC'] = '/usr/lib/ccache/gcc-11'

In [3]:
cuda_begin = """
//cuda
#include <torch/extension.h>
#include <stdio.h>
#include <c10/cuda/CUDAException.h>

#define CHECK_CUDA(x) TORCH_CHECK(x.device().is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)

inline unsigned int cdiv(unsigned int a, unsigned int b) { return (a + b - 1) / b;}
//!cuda
"""

## Problem 1

In this chapter we implemented a matrix multiplication kernel that has each thread produce one output matrix element. In this question, you will implement different matrix-matrix multiplication kernels and compare them.

a. Write a kernel that has each thread produce one output matrix row. Fill in the execution configuration parameters for the design.

In [15]:
cuda_src = cuda_begin + \
"""
//cuda
__global__ void matmul_row(float* m, float* n, float* out, int h, int w, int k) {
    int r = blockIdx.x*blockDim.x + threadIdx.x;

    if (r >= h) return;

    for (int c = 0; c < w; ++c) {
        float o = 0;
        for (int i = 0; i<k; ++i) {
            o += m[r*k + i] * n[i*w + c];
        }
        out[r*w+c] = o;
    }
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k == n.size(0), "Size mismatch!");
    auto output = torch::zeros({h, w}, m.options());

    dim3 tpb(256);
    dim3 blocks(cdiv(h, tpb.x));
    matmul_row<<<blocks, tpb>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
//!cuda
"""

cpp_src = \
"""
//cuda
torch::Tensor matmul(torch::Tensor m, torch::Tensor n);
//!cuda
"""

module = torch.utils.cpp_extension.load_inline(
    "test_ext", cpp_src, cuda_src,
    functions=['matmul'], extra_cuda_cflags=['--ptxas-options=-v'], verbose=True)

n = 32
A = torch.randn(n, n, device='cuda')
B = torch.randn(n, n, device='cuda')

A = torch.ones((3, 3), device='cuda')
B = torch.ones((3, 3), device='cuda')

out = module.matmul(A, B); torch.cuda.synchronize()
reference = torch.matmul(A, B)
print("Out:", out)
print("Reference:", reference)
print("Correct Implementation:", torch.allclose(out, reference))

import time
num_trials = 1_000

with torch.profiler.profile() as prof:
    for i in range(num_trials):
        module.matmul(A, B)
        torch.cuda.synchronize()

print(prof.key_averages().table())

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
No modifications detected for re-loaded extension module test_ext_v2, skipping build step...
Loading extension module test_ext_v2...


Out: tensor([[ -2.4887,  -1.7471,   2.4387,  ...,   0.1727,   1.1549,   1.4592],
        [ -3.4692,   8.6338,  -2.4712,  ...,   3.5144,   0.9575,   2.7203],
        [ -6.9637,   5.5027,   1.5716,  ...,   3.4485,   0.2339,   6.3301],
        ...,
        [  1.2553,  -0.8796,  -5.3700,  ...,   1.4417,   1.7112,   3.1901],
        [ 11.2950,   1.3072,  -3.6682,  ...,  10.2170,  -3.2171,  -1.7322],
        [-10.9152,  -0.5431,  -4.4932,  ...,  -6.4946,  -2.2655,  -9.4765]],
       device='cuda:0')
Reference: tensor([[ -2.4887,  -1.7471,   2.4387,  ...,   0.1727,   1.1549,   1.4592],
        [ -3.4692,   8.6338,  -2.4712,  ...,   3.5144,   0.9575,   2.7203],
        [ -6.9637,   5.5027,   1.5716,  ...,   3.4485,   0.2339,   6.3301],
        ...,
        [  1.2553,  -0.8796,  -5.3700,  ...,   1.4417,   1.7112,   3.1901],
        [ 11.2950,   1.3072,  -3.6682,  ...,  10.2170,  -3.2171,  -1.7322],
        [-10.9152,  -0.5431,  -4.4932,  ...,  -6.4946,  -2.2655,  -9.4765]],
       device='cuda:

b. Write a kernel that has each thread produce one output matrix column. Fill in the execution configuration parameters for the design.

In [5]:
cuda_src = cuda_begin + \
"""
//cuda
__global__ void matmul_col(float* m, float* n, float* out, int h, int w, int k) {
    int c = blockIdx.x*blockDim.x + threadIdx.x;

    if (c >= w) return;

    for (int r = 0; r < h; ++r) {
        float o = 0;
        for (int i = 0; i<k; ++i) {
            o += m[r*k + i] * n[i*w + c];
        }
        out[r*w+c] = o;
    }
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k == n.size(0), "Size mismatch!");
    auto output = torch::zeros({h, w}, m.options());

    dim3 tpb(256);
    dim3 blocks(cdiv(h, tpb.x));
    matmul_col<<<blocks, tpb>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
//!cuda
"""

cpp_src = \
"""
//cuda
torch::Tensor matmul(torch::Tensor m, torch::Tensor n);
//!cuda
"""

module = torch.utils.cpp_extension.load_inline(
    "test_ext", cpp_src, cuda_src,
    functions=['matmul'], extra_cuda_cflags=['--ptxas-options=-v'], verbose=True)

n = 32
A = torch.randn(n, n, device='cuda')
B = torch.randn(n, n, device='cuda')

# A = torch.ones((3, 3), device='cuda')
# B = torch.ones((3, 3), device='cuda')

out = module.matmul(A, B); torch.cuda.synchronize()
reference = torch.matmul(A, B)
print("Out:", out)
print("Reference:", reference)
print("Correct Implementation:", torch.allclose(out, reference))

import time
num_trials = 1_000

with torch.profiler.profile() as prof:
    for i in range(num_trials):
        module.matmul(A, B)
        torch.cuda.synchronize()

print(prof.key_averages().table())

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
The input conditions for extension module test_ext have changed. Bumping to version 1 and re-building as test_ext_v1...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/test_ext/build.ninja...
Building extension module test_ext_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module test_ext_v1...


Out: tensor([[ -0.0979,   6.7681,   0.6130,  ...,  -1.8185,   3.3314,  -2.8923],
        [  6.0187,   0.3002,  10.5269,  ...,   5.2424,  -6.9971,  -7.3472],
        [ -4.5366,   3.3448,  -6.6962,  ...,  -5.8891,   1.3179, -12.7183],
        ...,
        [ -1.6748,   1.6266, -16.4629,  ...,  -8.2513,  -6.5285,  -0.9741],
        [ -2.6126,  -2.9857,   8.3969,  ...,   7.6670,   7.5929,  -0.3515],
        [ 11.0854,  -2.4087,  -1.2288,  ...,  -7.0006,   5.3422,   5.4572]],
       device='cuda:0')
Reference: tensor([[ -0.0979,   6.7681,   0.6130,  ...,  -1.8185,   3.3314,  -2.8923],
        [  6.0187,   0.3002,  10.5269,  ...,   5.2424,  -6.9971,  -7.3472],
        [ -4.5366,   3.3448,  -6.6962,  ...,  -5.8891,   1.3179, -12.7183],
        ...,
        [ -1.6748,   1.6266, -16.4629,  ...,  -8.2513,  -6.5285,  -0.9741],
        [ -2.6126,  -2.9857,   8.3969,  ...,   7.6670,   7.5929,  -0.3515],
        [ 11.0854,  -2.4087,  -1.2288,  ...,  -7.0006,   5.3422,   5.4572]],
       device='cuda:

c. Analyze the pros and cons of each of the two kernel designs.

The pros & cons for the row wise and column wise matrix multiplication depends on the size of the matrices. Let A be of size (M x K), and B of size (K x N). If M > N, there are more rows than columns, so having the row-wise direction be paralelized is more beneficial so `matmul_row` is faster, and vice-versa.

## Problem 2

Write a matrix-vector multiplication kernel and the host stub function that can be called with four parameters: pointer to the output matrix, pointer to the input matrix, pointer to the input vector, and the number of elements in each dimension. Use one thread to calculate an output vector element.

In [6]:
cuda_src = cuda_begin + \
"""
//cuda
__global__ void matmul_kernel(float* m, float* n, float* out, int h, int w, int k) {
    int c = blockIdx.x*blockDim.x + threadIdx.x;
    int r = blockIdx.y*blockDim.y + threadIdx.y;

    if (c >= w || r >= h) return;

    float o = 0;
    for (int i = 0; i<k; ++i) {
        o += m[r*k + i] * n[i*w + c];
    }
    out[r*w+c] = o;
}

torch::Tensor matmul(torch::Tensor m, torch::Tensor n) {
    CHECK_INPUT(m); CHECK_INPUT(n);
    int h = m.size(0);
    int w = n.size(1);
    int k = m.size(1);
    TORCH_CHECK(k == n.size(0), "Size mismatch!");
    auto output = torch::zeros({h, w}, m.options());

    dim3 tpb(32, 32);
    dim3 blocks(cdiv(w, tpb.x), cdiv(h, tpb.y));
    matmul_kernel<<<blocks, tpb>>>(
        m.data_ptr<float>(), n.data_ptr<float>(), output.data_ptr<float>(), h, w, k);
    C10_CUDA_KERNEL_LAUNCH_CHECK();
    return output;
}
//!cuda
"""

cpp_src = \
"""
//cuda
torch::Tensor matmul(torch::Tensor m, torch::Tensor n);
//!cuda
"""

module = torch.utils.cpp_extension.load_inline(
    "test_ext", cpp_src, cuda_src,
    functions=['matmul'], extra_cuda_cflags=['--ptxas-options=-v'], verbose=True)

n = 32
A = torch.randn(n, n, device='cuda')
B = torch.randn(n, n, device='cuda')

# A = torch.ones((3, 3), device='cuda')
# B = torch.ones((3, 3), device='cuda')

out = module.matmul(A, B); torch.cuda.synchronize()
reference = torch.matmul(A, B)
print("Out:", out)
print("Reference:", reference)
print("Correct Implementation:", torch.allclose(out, reference))

import time
num_trials = 1_000

with torch.profiler.profile() as prof:
    for i in range(num_trials):
        module.matmul(A, B)
        torch.cuda.synchronize()

print(prof.key_averages().table())

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
The input conditions for extension module test_ext have changed. Bumping to version 2 and re-building as test_ext_v2...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/test_ext/build.ninja...
Building extension module test_ext_v2...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module test_ext_v2...


Out: tensor([[  2.1938,   7.5036,  -7.4128,  ...,   0.8692,   0.5169,   0.6550],
        [  1.0481,   7.5440,   4.3394,  ...,   4.4534,   2.5028,  -4.3398],
        [  6.1509,  -8.3925,   8.1037,  ..., -10.0252,   0.4850, -12.5438],
        ...,
        [  0.2771,  -4.8176, -11.1387,  ...,   3.7976,  -7.6448,   6.6561],
        [-11.1273,  14.3485,   2.7231,  ...,   0.8672,  -0.7542,  -3.8515],
        [  5.3164,  -0.9553,  10.9804,  ...,  -1.7326,   4.1123,   1.0089]],
       device='cuda:0')
Reference: tensor([[  2.1938,   7.5036,  -7.4128,  ...,   0.8692,   0.5169,   0.6550],
        [  1.0481,   7.5440,   4.3394,  ...,   4.4534,   2.5028,  -4.3398],
        [  6.1509,  -8.3925,   8.1037,  ..., -10.0252,   0.4850, -12.5438],
        ...,
        [  0.2771,  -4.8176, -11.1387,  ...,   3.7976,  -7.6448,   6.6561],
        [-11.1273,  14.3485,   2.7231,  ...,   0.8672,  -0.7542,  -3.8515],
        [  5.3164,  -0.9553,  10.9804,  ...,  -1.7326,   4.1123,   1.0089]],
       device='cuda:

## Problem 3

Consider the following CUDA kernel and the corresponding host function that calls it:

In [1]:
cuda_kernel = '''
//cuda
__global__ void foo_kernel(float* a, float* b, unsigned int M, unsigned int N) {
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
    unsigned int j = blockIdx.y * blockDim.y + threadIdx.y;
    if (i < M && j < N) {
        b[i * N + j] = a[i * N + j] / 2.1f + 4.8f;
    }
}

void foo(float* a_d, gloat* b_d) {
    unsigned int M = 150;
    unsigned int N = 300;
    dim3 bd(32, 32);
    dim3 gd((N-1)/16 + 1, (M-1)/32 + 1);
    foo_kernel<<<gd, bd>>>(a_d, b_d, M, N);
}
//!cuda
'''

What is the number of threads per block?
- There are `(floor((300-1)/16) + 1) * (floor((150-1)/32) + 1) = 95` threads per block

What is the number of threads in the grid?
- There are `95 * 32 * 32 = 97280` threads in the grid

What is the number of blocks in the grid?
- There are `32 * 32 = 1024` blocks in the grid.

What is the number of threads that execute the code on line 05?
- There are `150 * 300 = 45000` threads that execute the code on line 05.

Consider a 2D matrix with a width of 400 and a height of 500. The matrix is stored as a one-dimensional array. Specify the array index of the matrix element at row 20 and column 10:

If the matrix is stored in row-major order.
- The array index is `20 * 400 + 10 = 8010`

If the matrix is stored in column-major order.
- The array index is `10 * 500 + 20 = 5020`

Consider a 3D tensor with a width of 400, a height of 500, and a depth of 300. The tensor is stored as a one-dimensional array in row-major order. Specify the array index of the tensor element at x = 10, y = 20, and z = 5.
- The array index is `5 * 400 * 500 + 20 * 400 + 10 = 10002010`